<a href="https://colab.research.google.com/github/gackouhamady/TP1_Parquet_Avro/blob/main/Parquet_Avro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. **Bibliothec Needed**

In [ ]:
!pip install fastavro pandavro pyarrow python-snappy pandas

# 1.**Avro**

In [ ]:
import fastavro
import json

# Définition du schéma Avro
schema = {
    "type": "record",
    "name": "User",
    "fields": [
        {"name": "id", "type": "int"},
        {"name": "name", "type": "string"},
        {"name": "age", "type": "int"}
    ]
}

# Création de données
records = [
    {"id": 1, "name": "Alice", "age": 25},
    {"id": 2, "name": "Bob", "age": 30},
    {"id": 3, "name": "Charlie", "age": 22}
]

# Écriture dans un fichier Avro
with open("data.avro", "wb") as out_file:
    fastavro.writer(out_file, schema, records)


**Simple Read**

In [ ]:
# Lecture du fichier Avro
with open("data.avro", "rb") as in_file:
    reader = fastavro.reader(in_file)
    for record in reader:
        print(record)


**pandavro pour manipuler un DataFrame**

In [ ]:
import pandas as pd
import pandavro as pva

# Création d'un DataFrame Pandas
df = pd.DataFrame(records)

# Enregistrement au format Avro
pva.to_avro("data_pandas.avro", df)

# Chargement d'un fichier Avro en DataFrame
df_loaded = pva.read_avro("data_pandas.avro")
print(df_loaded)


# 2. **Parquet**

**Écriture d’un DataFrame en Parquet**

In [ ]:
import pyarrow.parquet as pq
import pyarrow as pa

# Conversion du DataFrame Pandas en Table Arrow
table = pa.Table.from_pandas(df)

# Enregistrement au format Parquet
pq.write_table(table, "data.parquet")


**Simple Read**

In [ ]:
# Lecture du fichier Parquet
table = pq.read_table("data.parquet")
df_parquet = table.to_pandas()
print(df_parquet)


**Sub column Readings**

In [ ]:
# Lecture d'un sous-ensemble des colonnes (ex: seulement 'name' et 'age')
df_subset = pq.read_table("data.parquet", columns=["name", "age"]).to_pandas()
print(df_subset)


#3. **Comparaison entre Avro et Parquet**

In [ ]:
import os

# Taille des fichiers
size_avro = os.path.getsize("data_pandas.avro")
size_parquet = os.path.getsize("data.parquet")

print(f"Taille du fichier Avro : {size_avro} octets")
print(f"Taille du fichier Parquet : {size_parquet} octets")


**Expérimentation avec la compression (Gzip vs Snappy)**

In [ ]:
pq.write_table(table, "data_gzip.parquet", compression="gzip")
size_gzip = os.path.getsize("data_gzip.parquet")
print(f"Taille du fichier Parquet avec Gzip : {size_gzip} octets")

**Compression avec Snappy**

In [ ]:
pq.write_table(table, "data_snappy.parquet", compression="snappy")
size_snappy = os.path.getsize("data_snappy.parquet")
print(f"Taille du fichier Parquet avec Snappy : {size_snappy} octets")